# Conclusion

Rappel de quelques concepts du parallélisme :

* Communications et synchronisations
    * Dépendances entre processeurs
* [Scalabilité](https://docs.alliancecan.ca/wiki/Scalability/fr) et
  [loi d'Amdahl](https://fr.wikipedia.org/wiki/Loi_d%27Amdahl)
    * Granularité - quantité minimale de travail à faire de manière autonome
    * Répartition de la charge

## MPI dans les autres langages

* Fortran : le standard MPI est déjà défini en Fortran
* C++ :
    * MPI 3.0 a éliminé les interfaces C++
    * [Boost MPI](https://www.boost.org/doc/libs/release/libs/mpi/) :
      bibliothèque pratique et très puissante
      pour les développeurs en C++
    * [Midi-conférence Boost-MPI](https://www.youtube.com/watch?v=U0axIKTO3wM)

```C++
boost::mpi::environment env(argc, argv);
boost::mpi::communicator world;
std::string s;

if (world.rank() == 0)
    world.recv(boost::mpi::any_source, 746, s);

```



## Défis de parallélisation supplémentaires

Les codes suivants fonctionnent déjà en mode séquentiel.
C'est maintenant à vous de les paralléliser avec MPI :

* [Écoulement de chaleur](https://github.com/calculquebec/cq-formation-ecoulement-chaleur)
* [Convolution sur une image](https://github.com/calculquebec/cq-formation-convolution/tree/main/defi-mpi)
* [Problème à N corps](https://github.com/calculquebec/cq-formation-nbody)

Pour mesurer le temps d’exécution d’un programme,
la commande `time srun` doit être dans un
[script de tâche](https://docs.alliancecan.ca/wiki/Running_jobs/fr#T.C3.A2che_MPI)
soumis avec la commande `sbatch` :

```Bash
#!/bin/bash
#SBATCH --ntasks=4
#SBATCH … # temps, mémoire, etc.

time srun ./programme
```

## Bonus - Notions avancées

MPI possède d'autres fonctionnalités :

* collectives à partitions inégales
* collectives non bloquantes (MPI 3)
* entrées-sorties parallèles (MPI-IO)
* **groupes, communicateurs, topologie**
* types dérivés
* accès distant (one-sided communication)
* communications persistantes
* processus dynamiques et fils

### Collectives à partitions inégales

* Les fonctions de communication collective vues jusqu'ici envoyaient
  des données de **même taille** pour chaque processus MPI
* Lorsque les données varient en taille, il faut dans certains
  cas utiliser des fonctions avec des arguments supplémentaires :
    * [MPI_Scatterv](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#page=252)
    * [MPI_Gatherv](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#page=242)
    * [MPI_Allgatherv](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#page=258)
    * [MPI_Alltoallv](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#page=262)

* Chaque processus a alors besoin de la position
  (**déplacement**) du premier élément qui le concerne et
  du nombre (**compte**) d'éléments à envoyer ou recevoir
* Tous les processus doivent calculer ces **déplacements**
  et **comptes** dans une étape préliminaire

`rank`                 |     0    |     1    |     2
:---------------------:|:--------:|:--------:|:--------:
vecteur (`n` éléments) | 5 8 3 12 | 34 12 45 | 43 65 2 1
Indices                | 0 1  2 3 |   4 5 6  | 7 8 9 10
déplacement (`debut`)  |     0    |     4    |     7
compte (`fin-debut`)   |     4    |     3    |     4